In [1]:
# These are the functions you need to read in the data. You need to have the datafiles in a data folder that is in
# the directory you are working in
#LINK IM USING FOR K-NN https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getData(name, attributes):
    i = 0
    dic = {}
    path = 'data/reviews_%s_5.json.gz' % name
    for line in parse(path):
        filtered = {}
        for k in line.keys():
            if k in attributes:
                filtered[k] = line[k]
            else:
                continue
        
        dic[i] = filtered
        i += 1
    return dic

In [2]:
# These are one of the three names you can choose to read the dataset from
names = ['Digital_Music', 'Kindle_Store', 'Video_Games']

# These are all the possible attributes that our datapoints can have. Lots of them are not that useful for us. So during
# the reading of the datafile you have to specify which of these attributes you want to include. For our purposes,
# only reviewerID, asin (product ID) and overall will be helpful. But I included them all, just in case.
attributes = ['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 
              'unixReviewTime', 'reviewTime']

In [3]:
# You read in the data for example like this:
data = getData('Digital_Music', ['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 
              'unixReviewTime', 'reviewTime'])

In [4]:
# This is what our data will look like. You have a dictionary with integers as keys [0,1,2,3,4...], and as values one 
# datapoint. Each datapoint in itself is a dictionary, with as keys the attribute, and as value the value of that attribute.
for i in range(20):
    print(data[i])

{'reviewerID': 'A3EBHHCZO6V2A4', 'asin': '5555991584', 'reviewerName': 'Amaranth "music fan"', 'helpful': [3, 3], 'reviewText': 'It\'s hard to believe "Memory of Trees" came out 11 years ago;it has held up well over the passage of time.It\'s Enya\'s last great album before the New Age/pop of "Amarantine" and "Day without rain." Back in 1995,Enya still had her creative spark,her own voice.I agree with the reviewer who said that this is her saddest album;it is melancholy,bittersweet,from the opening title song."Memory of Trees" is elegaic&majestic.;"Pax Deorum" sounds like it is from a Requiem Mass,it is a dark threnody.Unlike the reviewer who said that this has a "disconcerting" blend of spirituality&sensuality;,I don\'t find it disconcerting at all."Anywhere is" is a hopeful song,looking to possibilities."Hope has a place" is about love,but it is up to the listener to decide if it is romantic,platonic,etc.I\'ve always had a soft spot for this song."On my way home" is a triumphant endin

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import defaultdict

ds = pd.DataFrame.from_dict(data, orient='index')

print('Number of colums in Dataframe : ', len(ds.columns))
print('Number of rows in Dataframe : ', len(ds.index))
ds.head(n=10)

Number of colums in Dataframe :  9
Number of rows in Dataframe :  64706


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5.0,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5.0,Enya at her most elegant,991526400,"06 3, 2001"
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5.0,The best so far,1058140800,"07 14, 2003"
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5.0,Ireland produces good music.,957312000,"05 3, 2000"
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4.0,4.5; music to dream to,1200528000,"01 17, 2008"
5,A2P49WD75WHAG5,5555991584,Daniel J. Hamlow,"[62, 65]",Who knows why I initially considered this to b...,5.0,Dreamy unearthly magic continues with Eire's o...,1059523200,"07 30, 2003"
6,A3O90G1D7I5EGG,5555991584,dev1,"[1, 5]",Enya is one of a few artists whom I consider s...,3.0,Have You Hugged A Tree Today?,959385600,"05 27, 2000"
7,A3EJYJC25OJVKK,5555991584,"Distant Voyageur ""Nicholas Computer""","[5, 5]",Enya is one of of the most mysterious singers ...,5.0,The album for a romantic autumn evening,1040428800,"12 21, 2002"
8,A1DA8VOH9NR6C7,5555991584,E. Minkovitch,"[4, 4]",This is not another lousy Celtic New Age album...,5.0,Simply wonderful.,1167868800,"01 4, 2007"
9,A33TRNCQK4IUO7,5555991584,guillermoj,"[12, 12]","Many times, AND WITH GOOD REASON, the ""new age...",5.0,"A true gem, even if you don't love conventiona...",1052697600,"05 12, 2003"


In [18]:
ds_cleaned = ds.iloc[:,np.r_[:2, 5]]
ds_cleaned.head()

,reviewerID,asin,overall
0,A3EBHHCZO6V2A4,5555991584,5.0
1,AZPWAXJG9OJXV,5555991584,5.0
2,A38IRL0X2T4DPF,5555991584,5.0
3,A22IK3I6U76GX0,5555991584,5.0
4,A1AISPOIIHTHXX,5555991584,4.0


In [20]:
cleaned_val = ds_cleaned.values

user2product = defaultdict(dict)
for reviewerID, asin, overall in cleaned_val:
    user2product[reviewerID][asin] = overall

In [25]:
print("no. of users:", len(user2product))
print("no of reviews in user 1:", len(list(user2product.items())[0][1]))
print(list(user2product.items())[0])

no. of users: 5541
no of reviews in user 1: 91
('A3EBHHCZO6V2A4', {'5555991584': 5.0, 'B000002NJS': 5.0, 'B000002P5W': 5.0, 'B00000DHZJ': 5.0, 'B00005QZH2': 5.0, 'B00005YW4H': 2.0, 'B0009WFF0Y': 2.0, 'B000S50QYC': 5.0, 'B000V3L0ZK': 2.0, 'B000VS8UJY': 5.0, 'B000W0YVX0': 5.0, 'B000WL7UQ4': 2.0, 'B00137GGZC': 3.0, 'B00137KS2O': 3.0, 'B00137RGS8': 3.0, 'B0013D8BK4': 3.0, 'B0017OLUUC': 2.0, 'B001A7DDTC': 3.0, 'B001AAAT6O': 5.0, 'B001AAAT7I': 2.0, 'B001AAE9N8': 3.0, 'B001AAHQLA': 2.0, 'B001AUEMFS': 3.0, 'B001BNK58W': 2.0, 'B001DU4PHA': 3.0, 'B001FX5V8M': 3.0, 'B001FX9QS8': 1.0, 'B001GLHBIG': 4.0, 'B001IXQU3O': 3.0, 'B001IXSU8W': 5.0, 'B001IXUUNU': 5.0, 'B001JODA58': 3.0, 'B001K3JF7K': 2.0, 'B001L2BIHK': 2.0, 'B001NBO0KG': 2.0, 'B001Q6ERVA': 5.0, 'B001W3T44W': 3.0, 'B001X3EQLW': 2.0, 'B001XJTB8E': 3.0, 'B0024RI70M': 2.0, 'B00299CER2': 5.0, 'B002BPH1F4': 2.0, 'B002BPKWH8': 1.0, 'B002I53BL0': 5.0, 'B002K0OGOO': 3.0, 'B002O1QKGQ': 5.0, 'B002POXFQA': 5.0, 'B002X063LA': 5.0, 'B003DEE470': 1.0, 'B

[{}, {}, {}, {}, {}, {}]


In [8]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

NameError: name 'X_train' is not defined

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)